# 完成自己的 toy_git

## 初始化一个仓库

初始化本地 Git 仓库只需要创建 .git 目录及其下的一些文件和目录。在定义了 read_file 和 write_file 辅助函数之后，我们就可以编写init()

In [2]:
def init(repo):
    """Create directory for repo and initialize .git directory."""
    os.mkdir(repo)
    os.mkdir(os.path.join(repo, '.git'))
    for name in ['objects', 'refs', 'refs/heads']:
        os.mkdir(os.path.join(repo, '.git', name))
    write_file(os.path.join(repo, '.git', 'HEAD'),
               b'ref: refs/heads/master')
    print('initialized empty repository: {}'.format(repo))


你会注意到，没有很多优雅的错误处理。毕竟，这是一个 500 行的项目。如果 repo 目录已经存在，那么使用回溯就会失败。

## 散列对象

hash_object() 将对象散列并写入 `.git/objects`（“数据库”）。 Git 模型中有三种类型的对象：blob（普通文件），提交和树（这些表示单个目录的状态）。 每个对象都有一个小 header，包括字节的类型和大小。接下来是 NUL 字节，然后是文件的数据字节。所有文件都被 zlib 压缩并写入`.git/objects/ab/cd...`，其中 ab 是 40 个字符的 SHA-1 hash 的前两个字符，而 cd... 是剩余的字符的。 请注意使用Python 标准库（ os 和 hashlib）。

In [3]:
def hash_object(data, obj_type, write=True):
    """Compute hash of object data of given type and write to object store
    if "write" is True. Return SHA-1 object hash as hex string.
    """
    header = '{} {}'.format(obj_type, len(data)).encode()
    full_data = header + b'\x00' + data
    sha1 = hashlib.sha1(full_data).hexdigest()
    if write:
        path = os.path.join('.git', 'objects', sha1[:2], sha1[2:])
        if not os.path.exists(path):
            os.makedirs(os.path.dirname(path), exist_ok=True)
            write_file(path, zlib.compress(full_data))
    return sha1

然后是 find_object()，它通过散列（或散列前缀）找到一个对象。read_object() 读取一个对象及其类型 - 基本上是 hash_object() 的逆。最后，cat_file 是一个实现 git cat-file 的 pygit 等价的函数：它将对象的内容（或其大小或类型）打印到 stdout。

## Git 索引

我们接下来想做的事情就是将文件添加到索引或暂存区域中。索引是文件的入口，按照 path 排序，每个索引包含 path，修改时间，SHA-1 hash 等等。注意索引把当前树中的所有文件裂了出来，不仅仅是之前给 commit 缓存的文件。

索引是单文件在 `.git/index` 以自定义的二进制保存。并不是很复杂，但是为了得到可变长度路径之后的下一个索引条目确实需要一些 struct 上的用法以及加上一点点骚操作：

前 12 个字节是 header，最后 20 个字节是前面所有字节的 SHA-1 hash，中间的字节是索引的入口。这些索引是由 62 个字节以及加上路径的长度和一些填充值。

这里是 IndexEntry namedtuple 和 read_index()

In [5]:
import collections
# Data for one entry in the git index (.git/index)
IndexEntry = collections.namedtuple('IndexEntry', [
    'ctime_s', 'ctime_n', 'mtime_s', 'mtime_n', 'dev', 'ino', 'mode',
    'uid', 'gid', 'size', 'sha1', 'flags', 'path',
])

def read_index():
    """Read git index file and return list of IndexEntry objects."""
    try:
        data = read_file(os.path.join('.git', 'index'))
    except FileNotFoundError:
        return []
    digest = hashlib.sha1(data[:-20]).digest()
    assert digest == data[-20:], 'invalid index checksum'
    signature, version, num_entries = struct.unpack('!4sLL', data[:12])
    assert signature == b'DIRC', \
            'invalid index signature {}'.format(signature)
    assert version == 2, 'unknown index version {}'.format(version)
    entry_data = data[12:-20]
    entries = []
    i = 0
    while i + 62 < len(entry_data):
        fields_end = i + 62
        fields = struct.unpack('!LLLLLLLLLL20sH',
                               entry_data[i:fields_end])
        path_end = entry_data.index(b'\x00', fields_end)
        path = entry_data[fields_end:path_end]
        entry = IndexEntry(*(fields + (path.decode(),)))
        entries.append(entry)
        entry_len = ((62 + len(path) + 8) // 8) * 8
        i += entry_len
    assert len(entries) == num_entries
    return entries

此函数后跟 ls_files，status 和 diff 函数，所有这些函数实质上都是打印索引状态的不同方法

+ ls_file 只是打印在索引中的所有文件（如果 if -s 被指定的话还有它们的 mode 和 hash）


+ status 使用 get_status() 将索引中的文件与当前目录树中的文件进行比较，并打印出哪些文件被修改，建立和删除


+ diff 打印每个修改过的文件的差异，显示索引中的内容与当前工作副本中的内容（使用 Python 的 difflib）

考虑到文件修改时间和所有这些，我 100％ 肯定 git 对索引的使用和这些命令的实现比我的更有效。我只是通过 os.walk() 来列出完整目录列表，并使用一些集合的操作然后比较哈希。例如，这是我用来确定更改路径列表的集合理解


```python

changed = {p for p in (paths & entry_paths)
           if hash_object(read_file(p), 'blob', write=False) !=
              entries_by_path[p].sha1.hex()}
```

最后有一个 write_index 函数用于将索引写回，而 add() 用于向索引添加一个或多个路径 - 后者只是读取整个索引，添加路径，重新排序并再次写出来。

此时我们可以将文件添加到索引中，我们已准备好进行 commit。

## Commit

Commit 包括编写两个对象

首先，树对象，它是提交时当前目录（或实际上是索引）的快照。树只列出目录中文件（blob）和子树的哈希值 - 它是递归的。

所以每一个提交都是整个目录树的快照。但是通过 hash 保存数据的美妙之处在于如果在树中有任何文件发生了改变，整个树的 hash 也要发生改变。反过来，如果一个文件或者子树没有改变，hash 不会发生改变。所以你可以高效地存储树目录的变化。

这是一个由 cat-file pretty 2226 打印的树对象的示例（每行显示文件模式，对象类型，哈希和文件名）